In [1]:
import os, time, glob
import configparser
import inspect
import collections
from functools import singledispatch
import pandas as pd
import openpyxl

'''
基于工作日志导出月底交付件报表V.1.0
awx1192780
2022/09/15

conf.ini:
[smartwork]
base_folder = D:\\workspace
report_folder = D:\\workspace\\交付件
event = 【事件】
troubleshooting = 【故障】
pm = 【PM】
alarm = 【告警】
version_update = 【变更】
version_update_meeting = 【变更评审】
active_operation_maintain = 【主动运维】
faq = 【FAQ】
negative_event = 【负向事件】
'''
config = configparser.ConfigParser()
config.read('./conf.ini',encoding='UTF-8')
EVENT = str(config['smartwork']["event"])
TROUBLESHOOTING = str(config['smartwork']["troubleshooting"])
PM = str(config['smartwork']["pm"])
ALARM = str(config['smartwork']["alarm"])
VERSION_UPDATE = str(config['smartwork']["version_update"])
VERSION_UPDATE_MEETING = str(config['smartwork']["version_update_meeting"])
ACTIVE_OPERATION_MAINTAIN = str(config['smartwork']["active_operation_maintain"])
FAQ = str(config['smartwork']["faq"])
NEGATIVE_EVENT = str(config['smartwork']["negative_event"])
BASE_FOLDER = str(config['smartwork']["base_folder"])
REPORT_FOLDER = str(config['smartwork']["report_folder"])
keys_list = [EVENT,TROUBLESHOOTING,PM,ALARM,VERSION_UPDATE,VERSION_UPDATE_MEETING,ACTIVE_OPERATION_MAINTAIN,FAQ,NEGATIVE_EVENT]

In [2]:
def retrieve_name(var):
    return [var_name for var_name, var_val in inspect.currentframe().f_globals.items() if var_val == var]


def copy_dict_from(source_dict: dict):
    target_dict = collections.defaultdict(int)
    if isinstance(source_dict, dict):
        for i in source_dict.keys():
            target_dict[i] = 0
    return target_dict


def do_count(smart_dict, path):
    if '.txt' not in path:
        return None
    file = open(path, 'r', encoding='UTF-8')

    def verify(keys_list, line):
        for i in keys_list:
            if retrieve_name(i)[0] not in smart_dict:
                smart_dict[retrieve_name(i)[0]] = 0
            if line.find(i) != -1:
                return retrieve_name(i)[0]
    for line in file:
        # print(1)
        res = verify(keys_list, line)
        if res:
            smart_dict[res] += 1
    return smart_dict


def to_excel(smart_dict):
    try:
        df = pd.DataFrame([smart_dict.values()], columns=[
                          i.replace('【', '').replace('】', '') for i in keys_list])
        df.to_excel('{}\\{}交付件.xlsx'.format(REPORT_FOLDER, current_year),
                    sheet_name='{}月'.format(int(time.strftime('%m', time.localtime()))))
    except:
        print("导出失败，不存在目录或文件正在被使用。")


def traverse():
    current_year_and_month = time.strftime("%Y%m", time.localtime())
    current_year = time.strftime("%Y", time.localtime())
    smart_dict = collections.defaultdict(int)
    for dirname in os.listdir(BASE_FOLDER):
        dirpath = '{}\\{}'.format(BASE_FOLDER, dirname)
        if current_year_and_month in dirname and os.path.isdir(dirpath):
            for report in os.listdir(dirpath):
                if '.txt' in report:
                    file_path = '{}\\{}'.format(
                        dirpath, report).replace('\\', '\\\\')
                    # print(file_path)
                    smart_dict = do_count(smart_dict, file_path)


defaultdict(<class 'int'>, {})
